In [1]:
import requests
import os
import io
import zipfile

In [2]:
# Set workspace
os.chdir(r'E:\ArcGIS_2\Lab4')
wksp = os.getcwd()

## Minnesota boundary from MGC

In [3]:
# MN boundary
mn_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_state/shp_bdry_state.zip"
boundaries = requests.post(mn_url)
zipfile.ZipFile(io.BytesIO(boundaries.content)).extractall(wksp)

# Project shp
sr = arcpy.SpatialReference(4326)
arcpy.Project_management('Boundaries_of_Minnesota.shp', 'minnesota.shp', sr)

<Result 'E:\\ArcGIS_2\\Lab4\\minnesota.shp'>

## Stingbug observations

In [4]:
# Retrieve Stinkbugs Data
sbug_url = "https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/biota_bmsb/shp_biota_bmsb.zip"
sbug_data = requests.get(sbug_url, verify=True)
zipfile.ZipFile(io.BytesIO(sbug_data.content)).extractall(wksp)
sbug = "BMSBSurveyDataTable.dbf"

In [5]:
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprxMap = aprx.listMaps("Map")[0]

In [6]:
# Import stinkbug data table
sbugtable = arcpy.TableToTable_conversion(sbug, wksp, "sbugtable")

In [7]:
# Create point layer of data table
x_field = "Longitude"
y_field = "Latitude"
output_fc = "sbug"
spatial_reference = arcpy.SpatialReference(4326)
sbug_points = arcpy.management.XYTableToPoint(sbugtable, 
                                              output_fc, 
                                              x_field, 
                                              y_field, 
                                              "", 
                                              spatial_reference)

#### Verify points are within Minnesota boundary

In [8]:
# Select points that fall within the boundary of Minnesota and create new layer
selected_points = arcpy.SelectLayerByLocation_management("sbug", "INTERSECT", 'minnesota.shp')
MN_sbugpoints = arcpy.CopyFeatures_management(selected_points, "sbug_points.shp")

In [9]:
result = arcpy.GetCount_management("sbug_points.shp")
count = int(result.getOutput(0))

In [10]:
# Verify resulting layer only includes points within MN boundary
total_points = int(arcpy.GetCount_management("sbug_points.shp").getOutput(0))
if count == total_points:
    print("All points are within the boundary of Minnesota.")
else:
    print("Some points are outside the boundary of Minnesota.")

All points are within the boundary of Minnesota.


#### Verify observation counts are within acceptable range

In [11]:
# Set acceptable count range
acceptable_range = [0, 10000]

# Create empty lists to store invalid and valid observations
invalid_observations = []
valid_observations = []

# Look for valid/invalid observations and append them to their respective lists
with arcpy.da.SearchCursor("sbug_points.shp", ["Adults", "Nymphs"]) as cursor:
    for row in cursor:
        if row[0] >= acceptable_range[0] and row[0] <= acceptable_range[1]:
            # Add the valid observation to the list
            valid_observations.append(row)
            
            if row[1] >= acceptable_range[0] and row[1] <= acceptable_range[1]:
                # Add the valid observation to the list
                valid_observations.append(row)
            else:
                # Add the invalid observation to the list
                invalid_observations.append(row)
                
        else:
            # Add the invalid observation to the list
            invalid_observations.append(row)
            
print(len(invalid_observations))

0


## City boundaries

In [12]:
# Retrieve MN cities Data
territories_output = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/bdry_mn_city_township_unorg/shp_bdry_mn_city_township_unorg.zip')
zipfile.ZipFile(io.BytesIO(territories_output.content)).extractall(wksp)

In [13]:
# Project to WGS 1984
sr = arcpy.SpatialReference(4326)
arcpy.Project_management('city_township_unorg.shp', 'city_township_unorg_projected.shp', sr)

<Result 'E:\\ArcGIS_2\\Lab4\\city_township_unorg_projected.shp'>

In [14]:
# Extract only the cities
arcpy.analysis.Select(
    in_features="city_township_unorg_projected.shp",
    out_feature_class="cities.shp",
    where_clause="CTU_CLASS = 'CITY'"
)

<Result 'E:\\ArcGIS_2\\Lab4\\cities.shp'>

In [15]:
# Select only places with stink bug presence
arcpy.analysis.Select(
    in_features="sbug_points.shp",
    out_feature_class="sbug_presence.shp",
    where_clause="Adults <> 0 Or Nymphs <> 0"
)

<Result 'E:\\ArcGIS_2\\Lab4\\sbug_presence.shp'>

In [16]:
# Select the cities with stink bug presence
selection = arcpy.SelectLayerByLocation_management('city_township_unorg_projected.shp', "INTERSECT", 'sbug_presence.shp', 0, "NEW_SELECTION")
subselection = arcpy.SelectLayerByAttribute_management(selection, "SUBSET_SELECTION", "CTU_CLASS = 'CITY'")
arcpy.CopyFeatures_management(selection, 'sbug_cities.shp')

<Result 'E:\\ArcGIS_2\\Lab4\\sbug_cities.shp'>